In [107]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [9]:
close = pd.read_excel('./data/close.xlsx',header=1,index_col=0)
highest = pd.read_excel('./data/highest.xlsx',header=1,index_col=0)
industry = pd.read_excel('./data/industry.xlsx',header=1,index_col=0)
lowest = pd.read_excel('./data/lowest.xlsx',header=1,index_col=0)
marketcap = pd.read_excel('./data/marketcap.xlsx',header=1,index_col=0)
pct = pd.read_excel('./data/pct.xlsx',header=1,index_col=0)
shares = pd.read_excel('./data/shares.xlsx',header=1,index_col=0)
volume = pd.read_excel('./data/volume.xlsx',header=1,index_col=0)

In [10]:
date = marketcap.index[-1] # 取的最新的日期，有的指标直接展示了最新的值，所以会用

In [11]:
ret = close/close.shift(1)-1

In [82]:
# 涨跌幅提醒
pct_change_alert_1 = ret.applymap(lambda x: 1 if abs(x)>0.1 else 0)
pct_change_alert_2 = ret.shift(1).applymap(lambda x: 1 if abs(x)>0.1 else 0)
pct_change_alert = (pct_change_alert_1 ==1) & (pct_change_alert_2 ==1)
pct_change_alert = pct_change_alert.astype(int)
pct_change_alert.head()

,600000.SH,600004.SH,600006.SH,600007.SH,600008.SH,600009.SH,600010.SH,600011.SH,600012.SH,600015.SH,...,688787.SH,688788.SH,688789.SH,688793.SH,688798.SH,688799.SH,688800.SH,688819.SH,688981.SH,689009.SH
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2014-01-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2014-01-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2014-01-07,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2014-01-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
pct_change_alert.reset_index(inplace=True)
melted_pct_change_alert = pd.melt(pct_change_alert, id_vars=['Date'], value_vars=pct_change_alert.columns[2:], var_name='Company_name', value_name='Alert')
melted_pct_change_alert.head()

,Date,Company_name,Alert
0,2014-01-02,600000.SH,0
1,2014-01-03,600000.SH,0
2,2014-01-06,600000.SH,0
3,2014-01-07,600000.SH,0
4,2014-01-08,600000.SH,0


In [6]:
# 最新一天的涨跌幅提醒股票
date_result = pct_change_alert.loc[date,:]
date_result[date_result==1].index.tolist()

['603172.SH', '603879.SH', '603900.SH', '605488.SH']

In [101]:
# 二级行业收益率，市值加权
industry_new = industry.loc[date,:]
industry_list = list(set(list(industry_new)))

indus_ret = pd.DataFrame(index=ret.index, columns=industry_list)
for indus in industry_list:
    stock_list = list(industry_new[industry_new==indus].index)
    marketcap_weight = marketcap[stock_list].div(marketcap[stock_list].sum(axis=1), axis=0)
    marketcap_ret = ret[stock_list].mul(marketcap_weight.loc[date,:],axis=1)
    indus_ret.loc[:,indus] = marketcap_ret[stock_list].sum(axis=1)
indus_ret.head()

,种植业,一般零售,媒体,畜牧业,农产品加工Ⅱ,兵器兵装Ⅱ,专业市场经营Ⅱ,综合服务,农用化工,纺织制造,...,半导体,塑料及制品,装饰材料,航空航天,酒类,多领域控股Ⅱ,稀有金属,电气设备,石油化工,电商及服务Ⅱ
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-01-03,-0.022076,-0.009984,0.003682,-0.005318,-0.009371,0.000891,-0.019102,-0.016041,-0.017032,-0.008831,...,-0.000759,-0.000374,-0.001118,-0.010895,-0.013785,-0.020698,-0.00203,-0.001327,-0.0131,-0.003743
2014-01-06,-0.049471,-0.030269,-0.013701,-0.010687,-0.033179,-0.0185,-0.03344,-0.020328,-0.033333,-0.017688,...,-0.000636,-0.007025,-0.002823,-0.023672,-0.026983,-0.049073,-0.022389,-0.005561,0.009373,-0.005693
2014-01-07,0.00988,0.004006,0.008767,0.001654,0.003922,0.003487,-0.001913,0.001772,0.00163,0.003592,...,0.001082,0.004215,-0.000579,0.004069,0.000677,0.002003,-0.002425,0.020685,-0.015745,-0.001454
2014-01-08,-0.010576,-0.003621,0.011008,-0.002336,-0.009604,0.010373,0.004376,-0.007415,-0.010204,-0.005193,...,0.00034,-0.000881,0.0,0.014062,-0.011592,-0.005717,-0.011786,0.004953,-0.003965,-0.008769


In [62]:
industry_new.value_counts()

2024-04-03 00:00:00
半导体        116
其他化学制品Ⅱ    113
专用机械       110
其他医药医疗      89
汽车零部件Ⅱ      81
          ... 
专用材料Ⅱ        2
厨房电器Ⅱ        2
资产管理Ⅱ        1
新兴金融服务Ⅱ      1
林业           1
Name: count, Length: 107, dtype: int64

In [63]:
indus_vol = indus_ret.rolling(window=5).std()/indus_ret.rolling(window=5).mean()
indus_vol

,种植业,一般零售,媒体,畜牧业,农产品加工Ⅱ,兵器兵装Ⅱ,专业市场经营Ⅱ,综合服务,农用化工,纺织制造,...,半导体,塑料及制品,装饰材料,航空航天,酒类,多领域控股Ⅱ,稀有金属,电气设备,石油化工,电商及服务Ⅱ
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-08,-1.585966,-1.690972,4.997202,-1.460248,-1.494237,-14.300776,-1.582066,-1.153332,-1.209583,-1.467811,...,142.778352,-4.948569,-1.293420,-4.406701,-1.101656,-1.440650,-1.213761,2.715509,-2.168275,-0.882343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-28,-1.914612,-1.250152,-2.755066,-1.404145,-2.343154,-2.842612,-2.537235,-4.867200,-3.469828,-2.656620,...,-1.278294,-2.969643,-2.360339,-4.323372,-4.370184,-2.206883,-14.739274,-1.790485,-6.559480,-2.089554
2024-03-29,7.808782,-2.263532,-1.498400,77.987911,4.002350,-17.889028,-2.385513,-11.785162,5.055921,-24.094436,...,-1.803376,31.535303,-6.487196,-22.098829,-19.084466,-3.666035,1.856217,-5.452642,1.930792,-14.136626
2024-04-01,1.800360,6.330728,-11.825564,1.678483,1.126100,6.384071,5.502894,4.088328,1.493296,2.271806,...,-7.541824,2.188150,3.764376,6.642021,2.269160,20.518997,1.799987,5.143952,2.014875,3.222264


In [103]:
indus_vol.reset_index(inplace=True)
melted_indus_vol = pd.melt(indus_vol, id_vars=['Date'], value_vars=indus_vol.columns[2:], var_name='Industry_name', value_name='Volatility')
melted_indus_vol.head()

,Date,Industry_name,Volatility
0,2014-01-02,一般零售,NaN
1,2014-01-03,一般零售,NaN
2,2014-01-06,一般零售,NaN
3,2014-01-07,一般零售,NaN
4,2014-01-08,一般零售,-1.690972


In [102]:
indus_ret.reset_index(inplace=True)
melted_indus_ret = pd.melt(indus_ret, id_vars=['Date'], value_vars=indus_ret.columns[2:], var_name='Industry_name', value_name='Return')
melted_indus_ret.head()

,Date,Industry_name,Return
0,2014-01-02,一般零售,0.0
1,2014-01-03,一般零售,-0.009984
2,2014-01-06,一般零售,-0.030269
3,2014-01-07,一般零售,0.004006
4,2014-01-08,一般零售,-0.003621


In [10]:
# 最新一天波动率最大的五个板块
indus_vol.loc[date,:].nlargest(5)

资产管理Ⅱ    49.157311
通信设备     11.990134
黑色家电Ⅱ    10.386136
酒店及餐饮     9.227225
教育        8.583734
Name: 2024-04-03 00:00:00, dtype: float64

In [87]:
#集中性指标
indus_concentrate = pd.DataFrame(index=marketcap.index, columns=industry_list)

def top5_pct(row):
    top5_sum = row.nlargest(5).sum()
    return top5_sum/row.sum()

for indus in industry_list:
    stock_list = list(industry_new[industry_new==indus].index)
    indus_concentrate.loc[:,indus] = marketcap[stock_list].apply(top5_pct, axis=1)

indus_concentrate.head()

,种植业,一般零售,媒体,畜牧业,农产品加工Ⅱ,兵器兵装Ⅱ,专业市场经营Ⅱ,综合服务,农用化工,纺织制造,...,半导体,塑料及制品,装饰材料,航空航天,酒类,多领域控股Ⅱ,稀有金属,电气设备,石油化工,电商及服务Ⅱ
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-02,0.911996,0.476922,0.334532,0.57642,0.716639,0.909593,1.0,1.0,0.600071,0.433313,...,0.32612,0.682661,0.660092,0.581153,0.922697,1.0,0.659859,0.402131,0.968148,1.0
2024-01-03,0.911893,0.47595,0.334359,0.580853,0.718053,0.911811,1.0,1.0,0.601022,0.430502,...,0.328347,0.686538,0.660333,0.582295,0.92314,1.0,0.660409,0.40308,0.968999,1.0
2024-01-04,0.911825,0.473398,0.334821,0.580438,0.718149,0.909971,1.0,1.0,0.600967,0.432022,...,0.331147,0.683586,0.659236,0.581488,0.923419,1.0,0.659792,0.403514,0.969034,1.0
2024-01-05,0.912471,0.473247,0.335754,0.580434,0.719818,0.90988,1.0,1.0,0.602974,0.434184,...,0.330273,0.684593,0.659043,0.58315,0.924221,1.0,0.663667,0.405861,0.969051,1.0
2024-01-08,0.911785,0.468011,0.335578,0.580726,0.718972,0.909355,1.0,1.0,0.602597,0.434019,...,0.330583,0.687733,0.657645,0.585832,0.924752,1.0,0.667582,0.407983,0.969276,1.0


In [89]:
indus_concentrate.reset_index(inplace=True)
melted_indus_concentrate = pd.melt(indus_concentrate, id_vars=['Date'], value_vars=indus_concentrate.columns[2:], var_name='Industry_name', value_name='Concentrate')
melted_indus_concentrate.head()

,Date,Industry_name,Concentrate
0,2024-01-02,种植业,0.911996
1,2024-01-03,种植业,0.911893
2,2024-01-04,种植业,0.911825
3,2024-01-05,种植业,0.912471
4,2024-01-08,种植业,0.911785


In [95]:
#流动性总量
shares_num = shares.iloc[-1,:]  # 以最新股数为准吧，因为数据对不齐
turnover_rate_macro = volume.sum(axis=1)/shares_num.sum()
turnover_rate_macro = turnover_rate_macro.to_frame(name='Turnover_rate_macro')
turnover_rate_macro.head()

,Turnover_rate_macro
Date,
2024-01-02,0.524442
2024-01-03,0.492108
2024-01-04,0.456784
2024-01-05,0.519024
2024-01-08,0.466172


In [106]:
turnover_rate_change = turnover_rate_macro/turnover_rate_macro.shift(1)-1
turnover_rate_change.rename(columns={'Turnover_rate_macro': 'Turnover_rate_change'}, inplace=True)
turnover_rate_change.head()

,Turnover_rate_change
Date,
2024-01-02,NaN
2024-01-03,-0.061655
2024-01-04,-0.071781
2024-01-05,0.136256
2024-01-08,-0.101828


In [98]:
turnover_rate_indus = pd.DataFrame(index=volume.index, columns=industry_list)
for indus in industry_list:
    stock_list = list(industry_new[industry_new==indus].index)
    turnover_rate_indus.loc[:,indus] = volume[stock_list].sum(axis=1)/shares_num[stock_list].sum()
turnover_rate_indus.head()

,种植业,一般零售,媒体,畜牧业,农产品加工Ⅱ,兵器兵装Ⅱ,专业市场经营Ⅱ,综合服务,农用化工,纺织制造,...,半导体,塑料及制品,装饰材料,航空航天,酒类,多领域控股Ⅱ,稀有金属,电气设备,石油化工,电商及服务Ⅱ
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-02,0.682101,0.971335,1.319841,1.39342,1.191008,1.674447,0.396265,1.482221,0.844099,1.962394,...,0.620807,1.063129,2.558013,0.943156,1.210705,0.703279,0.689332,1.413229,0.210041,1.40447
2024-01-03,0.476,0.988173,1.627235,1.067855,1.080576,1.048059,0.313501,1.186873,1.337355,1.642957,...,0.654539,0.829097,2.117581,0.816831,0.939105,0.695346,0.578332,1.444811,0.184783,1.271471
2024-01-04,0.389083,0.861603,1.285237,0.752332,0.950016,1.140593,0.262652,1.028749,1.532552,1.492978,...,0.546913,0.742346,2.321048,0.677242,0.879133,0.749657,0.563361,1.245299,0.180323,1.035183
2024-01-05,0.450698,0.860196,1.179465,0.904104,1.094553,2.020455,0.322175,0.887597,1.514109,1.604379,...,0.572803,0.820271,2.612659,0.961811,0.765441,1.103545,0.827032,1.321822,0.18401,1.052577
2024-01-08,0.558839,1.156735,0.934494,1.847404,0.92966,1.071171,0.340222,0.932976,1.122721,1.309152,...,0.586056,1.072973,2.090555,0.721188,0.756599,1.067313,0.783599,1.566715,0.151525,1.150559


In [99]:
turnover_rate_indus.reset_index(inplace=True)
melted_turnover_rate_indus = pd.melt(turnover_rate_indus, id_vars=['Date'], value_vars=turnover_rate_indus.columns[2:], var_name='Industry_name', value_name='Turnover_rate')
melted_turnover_rate_indus.head()

,Date,Industry_name,Turnover_rate
0,2024-01-02,一般零售,0.971335
1,2024-01-03,一般零售,0.988173
2,2024-01-04,一般零售,0.861603
3,2024-01-05,一般零售,0.860196
4,2024-01-08,一般零售,1.156735


In [110]:
username = 'root'
password = 'QF5214'
host = '127.0.0.1'
database_name = 'group14'
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database_name}')
melted_pct_change_alert.to_sql(name='melted_pct_change_alert', con=engine, index=False, if_exists='replace')
melted_indus_vol.to_sql(name='melted_indus_vol', con=engine, index=False, if_exists='replace')
melted_indus_ret.to_sql(name='melted_indus_ret', con=engine, index=False, if_exists='replace')
melted_indus_concentrate.to_sql(name='melted_indus_concentrate', con=engine, index=False, if_exists='replace')
turnover_rate_macro.to_sql(name='turnover_rate_macro', con=engine, index=False, if_exists='replace')
turnover_rate_change.to_sql(name='turnover_rate_change', con=engine, index=False, if_exists='replace')
melted_turnover_rate_indus.to_sql(name='melted_turnover_rate_indus', con=engine, index=False, if_exists='replace')

6466